In [9]:
import cPickle as pickle
import matplotlib.pylab as pl
%matplotlib inline
import minpy.numpy as np
import minpy.nn.model_builder as builder
from minpy.core import grad_and_loss as _gradient_loss
from minpy.context import set_context, gpu, cpu
set_context(gpu(0))
import numpy as np0
from facility import *
from solver_primitives import *

In [7]:
def generate_noise(shape):
    return np.random.uniform(-1, 1, shape)

def d_gradient_loss(dmodel, gmodel, X):
    N, D = X.shape
    noise = generate_noise(X.shape)
    def _loss(*args):
        p_X = dmodel.forward(X, 'train')
        generated = gmodel.forward(noise, 'train')
        p_generated = dmodel.forward(generated, 'train')
        return np.sum(np.log(p_X) + np.log(1 - p_generated), axis=0) / float(N)
    gl = _gradient_loss(_loss, range(len(dmodel.params)))
    return gl(*dmodel.params.values())

def g_gradient_loss(dmodel, gmodel, X):
    N, D = X.shape
    noise = generate_noise(X.shape)
    def _loss(*args):
        generated = gmodel.forward(noise, 'train')
        p_generated = dmodel.forward(generated, 'train')
        return np.sum(np.log(1 - p_generated), axis=0) / float(N)
    gl = _gradient_loss(_loss, range(len(gmodel.params)))
    return gl(*gmodel.params.values())

In [8]:
ACTIVATION = 'ReLU'
activation = getattr(builder, ACTIVATION)

D = 2
DSHAPE = (D,) + (16,) * 4 + (1,)
dmlp = builder.Sequential()
for shape in DSHAPE[:-1]:
  dmlp.append(builder.Affine(shape))
  dmlp.append(activation())
dmlp.append(builder.Affine(DSHAPE[-1]))
dmodel = builder.Model(dmlp, 'l2', (D,))
initialize(dmodel)
dupdater = Updater(dmodel, 'sgd', {'learning_rate' : -0.01})

GSHAPE = (16,) * 4 + (D,)
gmlp = builder.Sequential()
for shape in GSHAPE[:-1]:
  gmlp.append(builder.Affine(shape))
  gmlp.append(activation())
gmlp.append(builder.Affine(GSHAPE[-1]))
gmodel = builder.Model(gmlp, 'l2', (D,))
initialize(gmodel)
gupdater = Updater(gmodel, 'sgd', {'learning_rate' : 0.01})

In [10]:
BATCH_SIZE = 1000
ITERATIONS = 1000
K = 1
INTERVAL = 10

X = pickle.load(open('sanity-data', 'rb'))
X_batches = Batches(X, BATCH_SIZE)
for i in range(ITERATIONS):
    for k in range(K):
        X_batch = next(X_batches)
        d_value, d_gradients = d_gradient_loss(dmodel, gmodel, X_batch)
        dupdater.update(d_gradients)
    g_value, g_gradients = g_gradient_loss(dmodel, gmodel, X_batch)
    gupdater.update(g_gradients)

TypeError: 'NoneType' object is not callable